### Back to Instance Properties

Let's try using `WeakKeyDictionary` to store our instance data in our data descriptor.

Basically, this is exactly the same as what we were doing before, but instead of using a standard dictionary (that potentially causes memory leaks), we'll use a `WeakKeyDictionary`.

Recall what we had before:

In [1]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

Now, we are going to refactor this to use the weak key dictionary:

In [2]:
import weakref

In [3]:
class IntegerValue:
    def __init__(self):
        self.values = weakref.WeakKeyDictionary()
        
    def __set__(self, instance, value):
        self.values[instance] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(instance)

And that's all there is to it. We now have weak references instead of strong references in our dictionary, and the dictionary cleans up after itself (removes "dead" entries) when the reference object has been destroyed by the GC.

In [4]:
class Point:
    x = IntegerValue()

In [5]:
p = Point()
print(hex(id(p)))

0x118653a60


In [6]:
p.x = 100.1

In [7]:
p.x

100

In [8]:
Point.x.values.keyrefs()

[<weakref at 0x118a36b60; to 'Point' at 0x118653a60>]

And if we delete `p`, thereby deleting the last strong reference to that object:

In [9]:
del p

In [10]:
Point.x.values.keyrefs()

[]

So this is almost a perfect general solution:

1. We do not need to store the data in the instances themseves (so we can handle objects whose class uses `__slots__`)
2. We are protected from memory leaks

But this only works for **hashable** objects.

So, now let's try to address this hashability issue.

Since we cannot use the object itself as the key in a dictionary (weak or otherwise), we could try using the `id` of the object (which is an int) as the key in a standard dictionary:

In [12]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = int(value)
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            return self.values.get(id(instance))

Now we can use this approach with non-hashable objects:

In [13]:
class Point:
    x = IntegerValue()
    
    def __init__(self, x):
        self.x = x
        
    def __eq__(self, other):
        return isinstance(other, Point) and self.x == other.x

In [14]:
p = Point(10.1)

In [15]:
p.x

10

In [16]:
p.x = 20.2

In [17]:
p.x

20

In [19]:
id(p), Point.x.values

(4704254608, {4704254608: 20})

Now we no longer have a memory leak:

In [20]:
import ctypes

def ref_count(address):
    return ctypes.c_long.from_address(address).value

In [21]:
p_id = id(p)

In [22]:
ref_count(p_id)

1

In [23]:
del p

In [24]:
ref_count(p_id)

0

But, we now have a "dead" entry in our dictionary - that memory address is still present as a key. Now, you might think it's not a big deal, but Python does reuse memory addresses, so we could run into potential issues there (where the data descriptor would have a value for a property already set from a previous object), and also the fact that our dictionary is cluttered with these dead entries:

In [25]:
Point.x.values

{4704254608: 20}

In [26]:
p = Point(10.1)

In [27]:
wP = weakref.ref(p)

In [28]:
ref_count(id(p))

1

In [29]:
del p

In [31]:
wP

<weakref at 0x1180f7ce0; dead>

So we need a way to determine if the object has been destroyed.

We know that weak references are aware of when objects are destroyed:

In [24]:
p = Point(10.1)
weak_p = weakref.ref(p)

In [25]:
print(hex(id(p)), weak_p)  
# again note how I need to use print to avoid affecting the ref count

0x7fa76043c588 <weakref at 0x7fa760439318; to 'Point' at 0x7fa76043c588>


In [26]:
ref_count(id(p))

1

And if I remove the last strong reference to `p`:

In [27]:
del p

In [28]:
print(weak_p)

<weakref at 0x7fa760439318; dead>


You can see that the weak reference was made aware of that change - in fact we can as well, by specifying a **callback** function that Python will call once the weak reference becomes dead (i.e. the object was destroyed by the GC):

In [32]:
def obj_destroyed(obj):
    print(f'{obj} is being destroyed')

In [33]:
p = Point(10.1)
w = weakref.ref(p, obj_destroyed)

In [34]:
del p

<weakref at 0x11881ba60; dead> is being destroyed


As you can see the callback function receives the weak ref object as the argument.

So, we can use this to our advantage in our data descriptor, by registering a callback that we can use to remove the "dead" entry from our values dictionary.

This means we do need to store a weak reference to the object as well - we'll do that in the value of the `values` dictionary as part of a tuple containing a weak reference to the object, and the corresponding value):

In [35]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), 
                                     int(value)
                                    )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]  # return the associated value, not the weak ref
        
    def _remove_object(self, weak_ref):
        print(f'removing dead entry for {weak_ref}')
        # how do we find that weak reference?

Let's just make sure our call back is being called as expected:

In [37]:
class Point:
    x = IntegerValue()

In [38]:
p1 = Point()
p2 = Point()

In [39]:
p1.x, p2.x = 10.1, 100.1

In [40]:
p1.x, p2.x

(10, 100)

Now let's delete those objects:

In [41]:
ref_count(id(p1)), ref_count(id(p2))

(1, 1)

In [42]:
del p1

removing dead entry for <weakref at 0x1186c4310; dead>


In [43]:
del p2

removing dead entry for <weakref at 0x118865710; dead>


OK, so now all that's left is to remove the corresponding entry from the dictionary. Problem is that we do not have the object itself at that point (and therefore do not have it's id either), so we cannot get to the dictionary item using the key - we'll simply have to iterate through the values in the dictionary until we find the value whose first item is the weak reference that caused the call back:

In [45]:
class IntegerValue:
    def __init__(self):
        self.values = {}
        
    def __set__(self, instance, value):
        self.values[id(instance)] = (weakref.ref(instance, self._remove_object), 
                                     int(value)
                                    )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.values.get(id(instance))
            return value_tuple[1]  # return the associated value, not the weak ref
        
    def _remove_object(self, weak_ref):
        reverse_lookup = [key for key, value in self.values.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.values[key]

In [46]:
class Point:
    x = IntegerValue()

In [47]:
p = Point()

In [48]:
p.x = 10.1

In [49]:
p.x

10

In [50]:
Point.x.values

{4700598384: (<weakref at 0x118a37560; to 'Point' at 0x1182d7070>, 10)}

Now let's delete our (only) strong reference to `p`:

In [51]:
ref_count(id(p))

1

In [52]:
del p

In [53]:
Point.x.values

{}

And as you can see our dictionary was cleaned up.

There is one last caveat, when we create weak references to objects, the weak reference objects are actually stored in the instance itself, in a property called `__weakref__`:

In [54]:
class Person:
    pass

In [55]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

Notice that `__weakref__` attribute. It is technically a data descriptor:

In [56]:
hasattr(Person.__weakref__, '__get__'), hasattr(Person.__weakref__, '__set__')

(True, True)

And instances will therefore have that property:

In [57]:
p = Person()

In [58]:
hasattr(p, '__weakref__')

True

In [59]:
print(p.__weakref__)

None


As you can see, that `__weakref__` attribute exists, but is currently `None`.

Now let's create a weak reference to `p`:

In [60]:
w = weakref.ref(p)

And `__weakref__` is no longer `None` (internally it is implemented as doubly linked list of all the weak references to that object - but this is an implementation detail and Python does not expose functionality to iterate through the weak references ourselves)

In [61]:
p.__weakref__

<weakref at 0x118864310; to 'Person' at 0x1182d68c0>

Now the problem if we use slots, is that the instances will no longer have that attribute!

In [62]:
class Person:
    __slots__ = 'name',

In [63]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name',),
              'name': <member 'name' of 'Person' objects>,
              '__doc__': None})

As you can see `__weakref__` is no longer an attribute in our class, and the instances do not have it:

In [64]:
p = Person()

In [65]:
hasattr(p, '__weakref__')

False

So, the problem is that we can no longer create weak references to this object!!

In [66]:
try:
    weakref.ref(p)
except TypeError as ex:
    print(ex)

cannot create weak reference to 'Person' object


In order to enable weak references in objects that use slots, we need to specify `__weakref__` as one of the slots:

In [67]:
class Person:
    __slots__ = 'name', '__weakref__'

In [68]:
Person.__dict__

mappingproxy({'__module__': '__main__',
              '__slots__': ('name', '__weakref__'),
              'name': <member 'name' of 'Person' objects>,
              '__weakref__': <attribute '__weakref__' of 'Person' objects>,
              '__doc__': None})

As you can see `__weakref__` is back, and exists in our instances:

In [69]:
p = Person()

In [70]:
hasattr(p, '__weakref__')

True

Which means we can create weak references to our `Person` object again:

In [71]:
w = weakref.ref(p)

So, if we want to use data descriptors using weak references (whether using our own dictionary or a weak key dictionary) with classes that define slots, we'll need to make sure we add `__weakref__` to the slots!

Let's do another example using this latest technique:

In [73]:
class ValidString:
    def __init__(self, min_length=0, max_length=255):
        self.data = {}
        self._min_length = min_length
        self._max_length = max_length
        
    def __set__(self, instance, value):
        if not isinstance(value, str):
            raise ValueError('Value must be a string.')
        if len(value) < self._min_length:
            raise ValueError(
                f'Value should be at least {self._min_length} characters.'
            )
        if len(value) > self._max_length:
            raise ValueError(
                f'Value cannot exceed {self._max_length} characters.'
            )
        self.data[id(instance)] = (weakref.ref(instance, self._finalize_instance), 
                                   value
                                  )
        
    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        else:
            value_tuple = self.data.get(id(instance))
            return value_tuple[1]  
        
    def _finalize_instance(self, weak_ref):
        reverse_lookup = [key for key, value in self.data.items()
                         if value[0] is weak_ref]
        if reverse_lookup:
            # key found
            key = reverse_lookup[0]
            del self.data[key]

We can now use `ValidString` as many times as we need:

In [74]:
class Person:
    __slots__ = '__weakref__',
    
    first_name = ValidString(1, 100)
    last_name = ValidString(1, 100)
    
    def __eq__(self, other):
        return (
            isinstance(other, Person) and 
            self.first_name == other.first_name and 
            self.last_name == other.last_name
        )
    
class BankAccount:
    __slots__ = '__weakref__',
    
    account_number = ValidString(5, 255)
    
    def __eq__(self, other):
        return (
            isinstance(other, BankAccount) and 
            self.account_number == other.account_number
        )

In [75]:
p1 = Person()

In [76]:
try:
    p1.first_name = ''
except ValueError as ex:
    print(ex)

Value should be at least 1 characters.


In [77]:
p2 = Person()

In [78]:
p1.first_name, p1.last_name = 'Guido', 'van Rossum'
p2.first_name, p2.last_name = 'Raymond', 'Hettinger'

In [79]:
b1, b2 = BankAccount(), BankAccount()

In [80]:
b1.account_number, b2.account_number = 'Savings', 'Checking'

In [81]:
p1.first_name, p1.last_name

('Guido', 'van Rossum')

In [82]:
p2.first_name, p2.last_name

('Raymond', 'Hettinger')

In [83]:
b1.account_number, b2.account_number

('Savings', 'Checking')

We can look at the data dictionary in each of the data descriptor instances:

In [84]:
Person.first_name.data

{4700600496: (<weakref at 0x118866bb0; to 'Person' at 0x1182d78b0>, 'Guido'),
 4700602080: (<weakref at 0x11887f0b0; to 'Person' at 0x1182d7ee0>, 'Raymond')}

In [85]:
Person.last_name.data

{4700600496: (<weakref at 0x111bb6570; to 'Person' at 0x1182d78b0>,
  'van Rossum'),
 4700602080: (<weakref at 0x11887dc10; to 'Person' at 0x1182d7ee0>,
  'Hettinger')}

In [86]:
BankAccount.account_number.data

{4700593344: (<weakref at 0x11864ea20; to 'BankAccount' at 0x1182d5cc0>,
  'Savings'),
 4700596944: (<weakref at 0x11887e0c0; to 'BankAccount' at 0x1182d6ad0>,
  'Checking')}

And if our objects are garbage collected:

In [87]:
del p1
del p2
del b1
del b2

In [88]:
Person.first_name.data

{}

In [89]:
Person.last_name.data

{}

In [84]:
BankAccount.account_number.data

{}

we can see that our dictionaries were cleaned up too!

OK, so this was a long journey, but it now allows us to handle classes that use slots and are not hashable. 

Depending on your needs, you may not need all this functionality (for example your objects may be guaranteed to be hashable and supports weak refs, in which case you can use the weak key dictionary approach), or maybe your class is guaranteed not to use slots (or contains `__dict__` as one of the slots), in which case you can just use the instance itself for storage (although the name to use is still an outstanding issue).

We'll circle back to using the instance for storage instead of using the data descripor itself in the next set of lectures.